## 8.3 语言模型和数据集
在8.2节中，我们了解了如何将文本数据映射为词元，以及将这些词元可以视为一系列离散的观测，例如单词或字符。假设长度为T的文本序列中的词元依次为$x_1, x_2, ..., x_T$。于是，$x_t (1 ≤ t ≤ T)$可以被认为是文本序列在时间步$t$处的观测或标签。在给定这样的文本序列时，语言模型(language model)的目标是估计序列的联合概率
$$
P(x_1, x_2, ..., x_T)
$$

例如，只需要一次抽取一个词元$x_t \sim P(x_t \mid x_{t-1}, ..., x_1)$，一个理想的语言模型就能够基于模型本身生成自然文本。与猴子使用打字机完全不同的是，从这样的模型中提取的文本豆浆作为自然语言(例如，英语文本)来传递。只需要基于前面的对话片断中的文本，就足以生成一个有意义的对话。显然，我们离设计出这样的系统还很遥远，因为它需要“理解”文本，而不仅仅是生成语法合理的内容。

尽管如此，语言模型依然是非常有用的。例如，短语“to recognize speech”和“to wreck a nice beach”读音上听起来非常相似。这种相似性会导致语音识别中的歧义，但是这很容易通过语言模型来解决，因为第二句的语义很奇怪。同样，在文档摘要生成算法中，“狗咬人”比“人咬狗”出现的频率要高得多，或者“我想吃奶奶”是一个相当匪夷所思的语句，而“我想吃，奶奶”则要正常多。

### 8.3.1 学习语言模型
显而易见，我们面对的问题是如何对一个文档，甚至一个词元序列进行建模。假设在单词级别对文本数据进行词元化，我们可以依靠在8.1节中对序列模型的分析。让我们从基本概率规则开始：
$$
P(x_1, x_2, ..., x_T) = \Pi_{t=1}^T P(x_t \mid x_1, ..., x_{t-1})
$$

例如，包含了四个单词的一个文本序列的概率是：
$$
 P(deep, learning, is, fun) = P(deep)P(learning \mid deep) P(is \mid deep, learning) P(fun \mid deep, learning, is)
$$

为了训练语言模型，我们需要计算单词的概率，以及给定前面几个单词后出现某个单词的条件概率。这些概率本质上就是语言模型的参数。

这里，我们假设训练数据集是一个大型的文本预料库。比如，维基百科的所有条目、古登堡计划，或者所有发布在网络上的文本。训练数据集中词的概率可以根据给定词的相对词频来计算。例如，可以讲估计值$\hat P(deep)$计算为任何以单词“deep”开头的句子的概率。一种方法是统计单词“deep”在数据集中出现的次数，然后将其除以整个语料库中的单词总数。这种方法效果不错，特别是对于频繁出现的单词。接下来，我们可以尝试估计
$$
\hat P(learning \mid deep) = \frac{n(deep, learning)}{n(deep)}
$$

其中$n(x)$和$n(x, x')$分别是单个单词和连续单词对的出现次数。不幸的是，由于连续单词对“deep learning”的出现频率要低得多，所以估计这类单词正确的概率要困难得多。特别是对于一些不常见的单词组合，要想找到足够的出现次数来获得准确的估计可能都不容易。而对于三个或者更多的单词组合，情况会变得更糟。许多合理的三个单词组合可能是存在的，但是在数据集中却找不到。除非我们提供某种解决方案，来将这些单词组合指定为非零计数，否则将无法在语言模型中使用它们。如果数据集很小，或者单词非常罕见，那么这类单词出现一次的机会可能都找不到。
一种常见的策略是执行某种形式的拉普拉斯平滑(Laplace smoothing)，具体方法是在所有计数中添加一个小常量。用$n$表示训练集中的单词总数，用$m$表示唯一单词的数量。此解决方案有助于处理单元素问题，例如通过：
$$
\hat P(x) = \frac{n(x) + \epsilon_1 /m}{n + \epsilon_1} \\
\hat P(x' \mid x) = \frac {n(x, x') + \epsilon_2 \hat P(x')}{n(x) + \epsilon_2}\\
\hat P(x'' \mid x, x') = \frac{n(x, x', x'') + \epsilon_3 \hat P(x'')} {n(x, x') + \epsilon_3}
$$